In [11]:
#Thêm dữ liệu và đọc file Excel
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('C:/Users/HOANG NAM/Desktop/Khai thác DL/lab2/bai3.csv')
df.head(10)

,Giới tính(GT),Tuổi(T),Năng lực làm việc (NL),Đã lập gia đình (LGD),Thu nhập (TN),Thăng chức (TC)
0,Nữ,20..25,Giỏi,Rồi,Rất cao,Có
1,Nam,20..25,Khá,Chưa,Khá,Không
2,Nữ,26..30,Giỏi,Chưa,Khá,Có
3,Nữ,31..40,T.Bình,Chưa,T.Bình,Có
4,Nam,26..30,T.Bình,Rồi,Rất cao,Không
5,Nữ,26..30,Khá,Chưa,Cao,Không
6,Nữ,31..40,Khá,Chưa,T.Bình,Không
7,Nam,26..30,Khá,Rồi,Cao,Có
8,Nữ,>40,Giỏi,Rồi,T.Bình,Không
9,Nữ,26..30,Giỏi,Chưa,Khá,Có


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
Giới tính(GT)             10 non-null object
Tuổi(T)                   10 non-null object
Năng lực làm việc (NL)    10 non-null object
Đã lập gia đình (LGD)     10 non-null object
Thu nhập (TN)             10 non-null object
Thăng chức (TC)           10 non-null object
dtypes: object(6)
memory usage: 608.0+ bytes


In [13]:
#Xóa Columns không cần thiết
df = df.drop(['Giới tính(GT)'],axis=1)
df = df.drop(['Đã lập gia đình (LGD)'],axis=1)
df = df.drop(['Thu nhập (TN)'],axis=1)

In [14]:
#Lấy giá trị từng dòng
records = []
for i in range((len(df))):
    records.append([str(df.values[i,j]) for j in range(0, df.shape[1])])
records[:10]

[['20..25', 'Giỏi', 'Có'],
 ['20..25', 'Khá ', 'Không'],
 ['26..30', 'Giỏi', 'Có'],
 ['31..40', 'T.Bình', 'Có'],
 ['26..30', 'T.Bình', 'Không'],
 ['26..30', 'Khá ', 'Không'],
 ['31..40', 'Khá ', 'Không'],
 ['26..30', 'Khá ', 'Có'],
 ['>40', 'Giỏi', 'Không'],
 ['26..30', 'Giỏi', 'Có']]

In [15]:
#Chuyển dữ liệu sang dạng encoder
from mlxtend.preprocessing.transactionencoder import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(records).transform(records)
df = pd.DataFrame(te_data, columns = te.columns_)
df

,20..25,26..30,31..40,>40,Có,Giỏi,Khá,Không,T.Bình
0,True,False,False,False,True,True,False,False,False
1,True,False,False,False,False,False,True,True,False
2,False,True,False,False,True,True,False,False,False
3,False,False,True,False,True,False,False,False,True
4,False,True,False,False,False,False,False,True,True
5,False,True,False,False,False,False,True,True,False
6,False,False,True,False,False,False,True,True,False
7,False,True,False,False,True,False,True,False,False
8,False,False,False,True,False,True,False,True,False
9,False,True,False,False,True,True,False,False,False


In [16]:
#Chạy giải thuật Apriori với min_su=0.2 và hiển thị tập phổ biến
from mlxtend.frequent_patterns import apriori

df1 = apriori(df, min_support=0.2, use_colnames = True, verbose=1)
df1

Processing 24 combinations | Sampling itemset size 3


,support,itemsets
0,0.2,(20..25)
1,0.5,(26..30)
2,0.2,(31..40)
3,0.5,(Có)
4,0.4,(Giỏi)
5,0.4,(Khá )
6,0.5,(Không)
7,0.2,(T.Bình)
8,0.3,"(26..30, Có)"
9,0.2,"(Giỏi, 26..30)"


In [17]:
#Tìm luật của tập phổ biến
from mlxtend.frequent_patterns import association_rules

rules = association_rules(df1, metric = "confidence", min_threshold = 0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Giỏi),(Có),0.4,0.5,0.3,0.75,1.5,0.1,2.0
1,(Khá ),(Không),0.4,0.5,0.3,0.75,1.5,0.1,2.0
2,"(Giỏi, 26..30)",(Có),0.2,0.5,0.2,1.00,2.0,0.1,inf


In [18]:
#Hiển thị các luật tìm được
rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
for i in range(len(rules)):
    print(rules.loc[i,'antecedents'],'==>',rules.loc[i,'consequents'],
         '[',rules.loc[i,'support'],',',rules.loc[i,'confidence'],']')

Giỏi ==> Có [ 0.3 , 0.7499999999999999 ]
Khá  ==> Không [ 0.3 , 0.7499999999999999 ]
Giỏi ==> Có [ 0.2 , 1.0 ]


In [19]:
#Chạy giải thuật FpGrowth với min_sup=0.2
from mlxtend.frequent_patterns import fpgrowth
itemsets = fpgrowth(df, min_support=0.2, use_colnames=True)
itemsets.head()

,support,itemsets
0,0.5,(Có)
1,0.4,(Giỏi)
2,0.2,(20..25)
3,0.5,(Không)
4,0.4,(Khá )


In [20]:
#Hiển thị các luật tìm được
rules = association_rules(itemsets, metric="confidence", min_threshold=0.7)
rules
rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)[0]).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: list(x)[0]).astype("unicode")
for i in range(len(rules)):
    print(rules.loc[i,'antecedents'],'==>',rules.loc[i,'consequents'],
         '[',rules.loc[i,'support'],',',rules.loc[i,'confidence'],']')

Giỏi ==> Có [ 0.3 , 0.7499999999999999 ]
Giỏi ==> Có [ 0.2 , 1.0 ]
Khá  ==> Không [ 0.3 , 0.7499999999999999 ]
